## Как выложить бота на HEROKU

*Подготовил Ян Пиле*

Сразу оговоримся, что мы на heroku выкладываем

**echo-Бота в телеграме, написанного с помощью библиотеки [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI)**.

А взаимодействие его с сервером мы сделаем с использованием [flask](http://flask.pocoo.org/)

То есть вы боту пишете что-то, а он вам отвечает то же самое.

## Регистрация

Идем к **@BotFather** в Telegram и по его инструкции создаем нового бота командой **/newbot**. 

Это должно закончиться выдачей вам токена вашего бота. Например последовательность команд, введенных мной:

* **/newbot**
* **my_echo_bot** (имя бота)
* **ian_echo_bot** (ник бота в телеграме)

Завершилась выдачей мне токена **1403467808:AAEaaLPkIqrhrQ62p7ToJclLtNNINdOopYk**

И ссылки  t.me/ian_echo_bot

<img src="botfather.png">

## Регистрация на HEROKU
 
Идем сюда: https://signup.heroku.com/login

Создаем пользователя (это бесплатно)

Попадаем на https://dashboard.heroku.com/apps и там создаем новое приложение:
<img src="newapp1.png">

Вводим название и регион (Я выбрал Европу), создаем.
<img src="newapp2.png">

После того, как приложение создано, нажмите, "Open App" и скопируйте адрес оттуда. 

<img src="newapp3.png">

У меня это https://ian-echo-bot.herokuapp.com

## Установить интерфейсы heroku и git для командной строки

Теперь надо установить Интерфейсы командной строки heroku и git по ссылкам:

* https://devcenter.heroku.com/articles/heroku-cli
* https://git-scm.com/book/en/v2/Getting-Started-Installing-Git

## Установить библиотеки

Теперь в вашем редакторе (например PyCharm) надо установить библиотеку для Телеграма и flask:

* pip install pyTelegramBotAPI
* pip install flask

## Код нашего echo-бота

Вот этот код я уложил в файл main.py

In [ ]:
import os
import telebot
from flask import Flask, request

TOKEN = '1403467808:AAEaaLPkIqrhrQ62p7ToJclLtNNINdOopYk' # это мой токен
bot = telebot.TeleBot(token=TOKEN)
server = Flask(__name__)

    
# Если строка на входе непустая, то бот повторит ее
@bot.message_handler(func=lambda msg: msg.text is not None)
def reply_to_message(message):
    bot.send_message(message.chat.id, message.text)

@server.route('/' + TOKEN, methods=['POST'])
def getMessage():
    bot.process_new_updates([telebot.types.Update.de_json(request.stream.read().decode("utf-8"))])
    return "!", 200

@server.route("/")
def webhook():
    bot.remove_webhook()
    bot.set_webhook(url='https://ian-echo-bot.herokuapp.com/' + TOKEN) #
    return "!", 200

if __name__ == "__main__":
    server.run(host="0.0.0.0", port=int(os.environ.get('PORT', 5000)))


## Теперь создаем еще два файла для запуска

**Procfile**(файл без расширения). Его надо открыть текстовым редактором и вписать туда строку:

    web: python main.py

**requirements.txt** - файл со списком версий необходимых библиотек. 

Зайдите в PyCharm, где вы делаете проект и введите в терминале команду:

    pip list format freeze > requirements.txt

В файле записи должны иметь вид:
    
    Название библиотеки   Версия библиотеки
    
Если вдруг вы выдите что-то такое:
<img src="versions.png">

Удалите этот кусок текста, чтоб остался только номер версии и сохраните файл.

Теперь надо все эти файлы уложить на гит, привязанный к Heroku и запустить приложение.



## Последний шаг

Надо залогиниться в heroku через командную строку.

Введите:

    heroku login

Вас перебросит в браузер на вот такую страницу:
<img src="login.png">


После того, как вы залогинились, удостоверьтесь, что вы находитесь в папке, где лежат фаши файлы:
    
    main.py
    Procfile
    requirements.txt

**Вводите команды:**

    git init 
    git add .
    git commit -m "first commit"
    heroku git:remote -a ian-echo-bot
    git push heroku master

По ходу выкатки вы увидите что-то такое:
<img src="process.png">
    

Готово, вы выложили вашего бота.

Материалы, которыми можно воспользоваться по ходу выкладки бота на сервер:

https://towardsdatascience.com/how-to-deploy-a-telegram-bot-using-heroku-for-free-9436f89575d2

https://mattrighetti.medium.com/build-your-first-telegram-bot-using-python-and-heroku-79d48950d4b0